# Subscriber

In [ ]:
import redis
from redis.commands.timeseries import TimeSeries
import paho.mqtt.client as mqtt
import json

# Redis connection setup
REDIS_HOST = 'redis-15780.c250.eu-central-1-1.ec2.redns.redis-cloud.com'
REDIS_PORT = 15780
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'DIZGajssLlzVl2kN1btFK0LKgBg36OLS'

redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)

# Verify that Redis TimeSeries is enabled
try:
    ts = redis_client.ts()
except AttributeError:
    raise RuntimeError("Redis TimeSeries module is not enabled. Please enable it in Redis.")

# Define the expected MAC address
expected_mac_address = "0xe45f01d8f49f"

mqtt_client = mqtt.Client()

def on_connect(mqtt_client, userdata, flags, rc):
    print(f'Connected with result code {str(rc)}')
    # Subscribe to all relevant topics
    mqtt_client.subscribe('s332203/#')

# Callback for receiving MQTT messages
def on_message(client, userdata, msg):
    try:
        # Decode the payload as JSON
        payload = json.loads(msg.payload.decode('utf-8'))
        print(f"Received message: {payload} on topic: {msg.topic}")

        # Check if the MAC address in the payload matches the expected one
        mac_address = payload.get("mac_address")
        if mac_address != expected_mac_address:
            print(f"Unknown MAC address: {mac_address}. Skipping storage.")
            return

        # Extract data from the payload
        timestamp = payload.get("timestamp", None)
        if timestamp is not None:
            # Convert the timestamp to milliseconds
            timestamp = int(float(timestamp) * 1000)
        else:
            timestamp = "*"  # Use "*" to indicate the current timestamp
        temperature = payload.get("temperature")
        humidity = payload.get("humidity")

        # Store data in Redis TimeSeries
        if temperature is not None:
            ts.add(f"{mac_address}:temperature", timestamp, temperature)
            print(f"Stored temperature {temperature} in Redis under {mac_address}:temperature.")

        if humidity is not None:
            ts.add(f"{mac_address}:humidity", timestamp, humidity)
            print(f"Stored humidity {humidity} in Redis under {mac_address}:humidity.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON payload: {msg.payload}")
    except Exception as e:
        print(f"Error processing message: {e}")

# Configure the MQTT client
mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_message

# MQTT broker parameters
mqtt_broker = "mqtt.eclipseprojects.io"
mqtt_port = 1883

# Connect to the broker
mqtt_client.connect(mqtt_broker, mqtt_port, 60)

# Start the MQTT loop to receive messages
print("MQTT subscriber is running and waiting for messages...")
mqtt_client.loop_forever()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5f0c18e4-6e8e-4b7e-be1d-d8fab55c80d7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>